In [1]:
from ff_espn_api import League
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keys import league_id, espn_s2, swid

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
!jt -t onedork -T -N -fs 105 -tfs 12 -nfs 125 -cellw 80% -lineh 125

In [4]:
# !jt -T -N -fs 105 -tfs 12 -nfs 125 -cellw 80% -lineh 125

In [5]:
# !jt -r

In [6]:
YEAR = 2019

In [7]:
REG_SEASON_END_WEEK = 13

In [8]:
# sets the current_week to 16 for completed seasons
if YEAR == 2019:
    CURRENT_WEEK = 9 # set this every week
else:
    CURRENT_WEEK = 16

In [9]:
# instatiate league object
# league_id, espn_s2, swid are in keys.py
league = League(league_id, YEAR, espn_s2, swid)

In [10]:
# creates league
# takes arg current_week, ignores it for a complete season
def create_league(year=YEAR, league=league, current_week=CURRENT_WEEK):
    # get a list of columns for our dataframe
    df_columns = list(league.teams[0].__dict__.keys())
    # remove roster from list of columns
    df_columns.remove('roster')
    
    # instantiate empty dataframe
    league_year = pd.DataFrame()
    
    # append teams to dataframe
    for d in range(len(league.teams)):
        team_df = pd.DataFrame(league.teams[d].__dict__, columns=df_columns)
        league_year = league_year.append(team_df)
    
    
    # add columns
    league_year['delta_points'] = league_year['points_for'] - league_year['points_against']
    leauge_year = week_and_year(league_year)
    league_year = draft_order(league_year)
    league_year = cumsum_score(league_year)
    league_year = overall_win_loss(league_year) # must be last
    
    return league_year

In [11]:
# creates year and week column
def week_and_year(league_year, year=YEAR):
    league_year['year'] = year    # add a year column
    league_year.index.names = ['week']    # change index to 'week'
    
    
    league_year.reset_index(level='week', inplace=True)
    league_year['week'] = league_year['week'].apply(lambda x: x + 1)
    return league_year

In [12]:
# create draft order column
def draft_order(league_year, league=league):
    # instantiate empty dictionaries
    team_dict = {}
    league_dict = {}
    
    # create dictionary of {team name: draft order number}
    for draft_number in range(1, len(league.teams)+1):
        team_name = str(league.draft[draft_number-1].team) # convert to string
        team_name = team_name[5:-1] #eliminates .strip() 'team' problem
        team_dict = {team_name: draft_number}
        league_dict.update(team_dict)
        
    league_year['draft_order'] = league_year['team_name'].map(league_dict)
    
    return league_year

In [13]:
# adds *4* columns for overall win/loss for entire league
def overall_win_loss(league_year, current_week=CURRENT_WEEK, reg_season_end_week=REG_SEASON_END_WEEK, league=league):
    # set null values for columns
    league_year['weekly_OW'] = np.nan
    league_year['weekly_OL'] = np.nan
    league_year['season_OW'] = np.nan
    league_year['season_OL'] = np.nan
    
    # weekly_OW/OL columns
    for week in range(current_week + 1):
        # playoffs start at week 13
        if week <= reg_season_end_week:
            win = len(league.teams) - 1
            loss = 0
            # sets W/L for each team
            for i, row in league_year[league_year['week'] == week].sort_values('scores', ascending=False).iterrows():
                league_year['weekly_OW'].iloc[i] = win
                league_year['weekly_OL'].iloc[i] = loss
                win -= 1
                loss += 1
        # values remain the same during playoffs
        else:
            win = 0
            loss = 0
            # sets W/L for each team
            for i, row in league_year[league_year['week'] == week].sort_values('scores', ascending=False).iterrows():
                league_year['weekly_OW'].iloc[i] = win
                league_year['weekly_OL'].iloc[i] = loss
        print(f'Week {week + 1} complete')
    
    # season_OW/OL columns
    for team in league_year['team_id'].unique():
        season_wins = 0
        season_loss = 0
        
        # sets W/L for each team
        for i, row in league_year[league_year['team_id'] == team].sort_values('week').iterrows():
            season_wins = season_wins + row[-4]
            season_loss = season_loss + row[-3]
            league_year['season_OW'].iloc[i] = season_wins
            league_year['season_OL'].iloc[i] = season_loss
        print(f'Team {team} complete')
    
    
    return league_year

In [14]:
def cumsum_score(league_year, reg_season_end_week=REG_SEASON_END_WEEK):
    league_year['cumsum_score'] = np.nan
    for team in league_year['team_id'].unique():
        cumulative = 0
        for i, row in league_year[league_year['team_id'] == team].iterrows():
            if league_year['week'].iloc[i] <= reg_season_end_week:
                cumulative = cumulative + league_year['scores'].iloc[i]
            league_year['cumsum_score'].iloc[i] = cumulative

    return league_year

In [15]:
league_2019 = create_league()

Week 1 complete
Week 2 complete
Week 3 complete
Week 4 complete
Week 5 complete
Week 6 complete
Week 7 complete
Week 8 complete
Week 9 complete
Week 10 complete
Team 1 complete
Team 2 complete
Team 3 complete
Team 4 complete
Team 5 complete
Team 6 complete
Team 7 complete
Team 8 complete
Team 9 complete
Team 10 complete
Team 11 complete
Team 12 complete


In [16]:
league_2019

,week,team_id,team_abbrev,team_name,division_id,wins,losses,points_for,points_against,owner,...,outcomes,mov,delta_points,year,draft_order,cumsum_score,weekly_OW,weekly_OL,season_OW,season_OL
0,1,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,HOME,29.22,-25.98,2019,7,118.14,7.0,4.0,7.0,4.0
1,2,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,AWAY,-6.77,-25.98,2019,7,208.78,3.0,8.0,10.0,12.0
2,3,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,HOME,-20.34,-25.98,2019,7,308.16,3.0,8.0,13.0,20.0
3,4,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,HOME,-0.66,-25.98,2019,7,387.96,2.0,9.0,15.0,29.0
4,5,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,AWAY,-17.98,-25.98,2019,7,510.84,8.0,3.0,23.0,32.0
5,6,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,AWAY,-10.70,-25.98,2019,7,578.64,1.0,10.0,24.0,42.0
6,7,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,AWAY,9.31,-25.98,2019,7,667.70,5.0,6.0,29.0,48.0
7,8,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,HOME,-1.34,-25.98,2019,7,759.91,1.0,10.0,30.0,58.0
8,9,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,HOME,-6.72,-25.98,2019,7,865.71,5.0,6.0,35.0,64.0
9,10,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,UNDECIDED,0.00,-25.98,2019,7,865.71,NaN,NaN,NaN,NaN


In [17]:
# up until this week
league_2019[league_2019['week'] <= CURRENT_WEEK]

,week,team_id,team_abbrev,team_name,division_id,wins,losses,points_for,points_against,owner,...,outcomes,mov,delta_points,year,draft_order,cumsum_score,weekly_OW,weekly_OL,season_OW,season_OL
0,1,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,HOME,29.22,-25.98,2019,7,118.14,7.0,4.0,7.0,4.0
1,2,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,AWAY,-6.77,-25.98,2019,7,208.78,3.0,8.0,10.0,12.0
2,3,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,HOME,-20.34,-25.98,2019,7,308.16,3.0,8.0,13.0,20.0
3,4,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,HOME,-0.66,-25.98,2019,7,387.96,2.0,9.0,15.0,29.0
4,5,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,AWAY,-17.98,-25.98,2019,7,510.84,8.0,3.0,23.0,32.0
5,6,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,AWAY,-10.70,-25.98,2019,7,578.64,1.0,10.0,24.0,42.0
6,7,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,AWAY,9.31,-25.98,2019,7,667.70,5.0,6.0,29.0,48.0
7,8,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,HOME,-1.34,-25.98,2019,7,759.91,1.0,10.0,30.0,58.0
8,9,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,HOME,-6.72,-25.98,2019,7,865.71,5.0,6.0,35.0,64.0
13,1,2,DPRK,kor ea,1,1,8,916.89,1085.55,Patrick Davis,...,HOME,-29.22,-168.66,2019,3,88.92,2.0,9.0,2.0,9.0


In [18]:
# current rankings, sorted by division
league_2019[league_2019['week'] == CURRENT_WEEK].sort_values(by=['division_id', 'wins'], ascending=[True, False])

,week,team_id,team_abbrev,team_name,division_id,wins,losses,points_for,points_against,owner,...,outcomes,mov,delta_points,year,draft_order,cumsum_score,weekly_OW,weekly_OL,season_OW,season_OL
34,9,3,TEV,SAINt TEV,0,6,3,1009.92,1017.32,Tevin Cropper,...,AWAY,5.75,-7.40,2019,2,1009.92,4.0,7.0,62.0,37.0
112,9,9,d:),"Take Mahomes, Country Road.",0,6,3,882.14,845.68,Eric Cudiamat,...,AWAY,7.16,36.46,2019,6,882.14,2.0,9.0,43.0,56.0
86,9,7,VERH,My Ball Zach Ertz,0,5,4,973.99,1002.97,Michael Verhulst,...,HOME,-2.24,-28.98,2019,10,973.99,7.0,4.0,56.0,43.0
125,9,10,WATY,YMBS THOTBOI,1,4,5,963.91,906.23,Gary Watson,...,AWAY,67.57,57.68,2019,4,963.91,8.0,3.0,56.0,43.0
8,9,1,JAMP,Dennis Jonman,1,2,7,865.71,891.69,Jon Picart,...,HOME,-6.72,-25.98,2019,7,865.71,5.0,6.0,35.0,64.0
21,9,2,DPRK,kor ea,1,1,8,916.89,1085.55,Patrick Davis,...,HOME,-13.31,-168.66,2019,3,916.89,10.0,1.0,47.0,52.0
151,9,12,PICA,dick chubb,2,7,2,955.29,828.69,dj Picart,...,HOME,6.72,126.60,2019,11,955.29,6.0,5.0,57.0,42.0
73,9,6,MEC,Game of Jones,2,6,3,878.39,837.16,Marc Cudiamat,...,AWAY,-5.75,41.23,2019,1,878.39,3.0,8.0,43.0,56.0
47,9,4,PARA,YUH OUU LIL PUMP,2,3,6,907.56,955.79,Kevin Paragna,...,HOME,13.31,-48.23,2019,12,907.56,11.0,0.0,45.0,54.0
99,9,8,CJL,AndyReids Above Ground Sex Pool,3,6,3,985.97,890.16,Cody Leonard,...,HOME,2.24,95.81,2019,5,985.97,9.0,2.0,57.0,42.0


In [19]:
league_2019[league_2019['week'] == CURRENT_WEEK].sort_values('season_OW', ascending=False)

,week,team_id,team_abbrev,team_name,division_id,wins,losses,points_for,points_against,owner,...,outcomes,mov,delta_points,year,draft_order,cumsum_score,weekly_OW,weekly_OL,season_OW,season_OL
34,9,3,TEV,SAINt TEV,0,6,3,1009.92,1017.32,Tevin Cropper,...,AWAY,5.75,-7.40,2019,2,1009.92,4.0,7.0,62.0,37.0
99,9,8,CJL,AndyReids Above Ground Sex Pool,3,6,3,985.97,890.16,Cody Leonard,...,HOME,2.24,95.81,2019,5,985.97,9.0,2.0,57.0,42.0
151,9,12,PICA,dick chubb,2,7,2,955.29,828.69,dj Picart,...,HOME,6.72,126.60,2019,11,955.29,6.0,5.0,57.0,42.0
86,9,7,VERH,My Ball Zach Ertz,0,5,4,973.99,1002.97,Michael Verhulst,...,HOME,-2.24,-28.98,2019,10,973.99,7.0,4.0,56.0,43.0
125,9,10,WATY,YMBS THOTBOI,1,4,5,963.91,906.23,Gary Watson,...,AWAY,67.57,57.68,2019,4,963.91,8.0,3.0,56.0,43.0
60,9,5,GOAT,Hit the SaQuon,3,5,4,936.30,907.31,Kuljeet Sidhu,...,AWAY,-7.16,28.99,2019,9,936.30,1.0,10.0,53.0,46.0
21,9,2,DPRK,kor ea,1,1,8,916.89,1085.55,Patrick Davis,...,HOME,-13.31,-168.66,2019,3,916.89,10.0,1.0,47.0,52.0
47,9,4,PARA,YUH OUU LIL PUMP,2,3,6,907.56,955.79,Kevin Paragna,...,HOME,13.31,-48.23,2019,12,907.56,11.0,0.0,45.0,54.0
73,9,6,MEC,Game of Jones,2,6,3,878.39,837.16,Marc Cudiamat,...,AWAY,-5.75,41.23,2019,1,878.39,3.0,8.0,43.0,56.0
112,9,9,d:),"Take Mahomes, Country Road.",0,6,3,882.14,845.68,Eric Cudiamat,...,AWAY,7.16,36.46,2019,6,882.14,2.0,9.0,43.0,56.0


In [20]:
columns = ['owner', 'team_abbrev', 'team_name', 'week', 'points_for', 'points_against', 'delta_points', 'draft_order', 'season_OW', 'season_OL']

In [21]:
specific = league_2019[columns]

In [22]:
specific[specific['week'] == CURRENT_WEEK].sort_values('season_OW', ascending=False)

,owner,team_abbrev,team_name,week,points_for,points_against,delta_points,draft_order,season_OW,season_OL
34,Tevin Cropper,TEV,SAINt TEV,9,1009.92,1017.32,-7.40,2,62.0,37.0
99,Cody Leonard,CJL,AndyReids Above Ground Sex Pool,9,985.97,890.16,95.81,5,57.0,42.0
151,dj Picart,PICA,dick chubb,9,955.29,828.69,126.60,11,57.0,42.0
86,Michael Verhulst,VERH,My Ball Zach Ertz,9,973.99,1002.97,-28.98,10,56.0,43.0
125,Gary Watson,WATY,YMBS THOTBOI,9,963.91,906.23,57.68,4,56.0,43.0
60,Kuljeet Sidhu,GOAT,Hit the SaQuon,9,936.30,907.31,28.99,9,53.0,46.0
21,Patrick Davis,DPRK,kor ea,9,916.89,1085.55,-168.66,3,47.0,52.0
47,Kevin Paragna,PARA,YUH OUU LIL PUMP,9,907.56,955.79,-48.23,12,45.0,54.0
73,Marc Cudiamat,MEC,Game of Jones,9,878.39,837.16,41.23,1,43.0,56.0
112,Eric Cudiamat,d:),"Take Mahomes, Country Road.",9,882.14,845.68,36.46,6,43.0,56.0
